In [1]:

from Guwahati import Guwahati

nwh = Guwahati.create_hierarchy()

print(nwh.pprint() )

=== NW Hierarchy Pretty Print ===
📍 NW_root (ID: -1, Type: NW_root)
     🏢 Nijeshwari PWSS (ID: 97, Type: location)
          💧 Abstraction (ID: 99, Type: water_abstraction)
               🏔️ Source (ID: 100, Type: source_module)
                    🔧 Borewell level (ID: 69, Type: level, Primary: groundwater_level)
                         📦 D1004901122 (ID: 79, Product: FMX21)
                    🔧 Power Mains (ID: 70, Type: power, Primary: total_kwh)
                         📦 2407811548 (ID: 87, Product: Energy meter)
                    🔧 ABS-Pump (ID: 71, Type: pump, Primary: abs_pump_on)
                         📦 Guwahati_ABS-Pump01 (ID: 88, Product: Pump)
          🚰 Distribution (ID: 101, Type: water_distribution)
               🧽 Treated water (ID: 102, Type: disinfection_module)
                    🔧 Disinfect Transmitter (ID: 73, Type: undefined, Primary: None)
                         📦 L70E7D05G00 (ID: 80, Product: CM442)
                    🔧 Chlorine (ID: 75, Type: anal

In [2]:
type(-1)

int

In [2]:
nwh.get_node_counts()

{'locations': 1,
 'applications': 2,
 'modules': 4,
 'instrumentations': 14,
 'assets': 14,
 'total': 35}

In [3]:
nwh.nodes[82].thresholds

[{'name': 'Active', 'key': 'volumeflow', 'type': 'high', 'value': 72.0}]